In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import os
Root = '/content/drive/MyDrive/SER21-7'
os.chdir(Root)

In [ ]:
ls

Actor_01/  Actor_05/  Actor_09/  Actor_13/  Actor_17/  Actor_21/  modelForPrediction1.sav
Actor_02/  Actor_06/  Actor_10/  Actor_14/  Actor_18/  Actor_22/  SER.ipynb
Actor_03/  Actor_07/  Actor_11/  Actor_15/  Actor_19/  Actor_23/
Actor_04/  Actor_08/  Actor_12/  Actor_16/  Actor_20/  Actor_24/


In [ ]:
import librosa
import soundfile
import os, glob, pickle
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import accuracy_score
from IPython.display import Audio  # For audio playback

In [ ]:
# Extract features (mfcc, chroma, mel) from a sound file
def extract_feature(file_name, mfcc, chroma, mel):
    with soundfile.SoundFile(file_name) as sound_file:
        X = sound_file.read(dtype="float32")
        sample_rate = sound_file.samplerate
        if chroma:
            stft = np.abs(librosa.stft(X))
        result = np.array([])
        if mfcc:
            mfccs = np.mean(librosa.feature.mfcc(y=X, sr=sample_rate, n_mfcc=40).T, axis=0)
            result = np.hstack((result, mfccs))
        if chroma:
            chroma = np.mean(librosa.feature.chroma_stft(S=stft, sr=sample_rate).T, axis=0)
            result = np.hstack((result, chroma))
        if mel:
            mel = np.mean(librosa.feature.melspectrogram(y=X, sr=sample_rate).T, axis=0)
            result = np.hstack((result, mel))
    return result


In [ ]:
# Emotions in the RAVDESS dataset
emotions = {
    '01': 'neutral',
    '02': 'calm',
    '03': 'happy',
    '04': 'sad',
    '05': 'angry',
    '06': 'fearful',
    '07': 'disgust',
    '08': 'surprised'
}

In [ ]:
# Emotions to observe
observed_emotions = ['calm', 'happy', 'fearful', 'disgust']

In [ ]:
# Load the data and extract features for each sound file
def load_data(test_size=0.2):
    x, y = [], []
    for file in glob.glob("/content/drive/MyDrive/SER21-7/Actor_*/*.wav"):
        file_name = os.path.basename(file)
        emotion = emotions[file_name.split("-")[2]]
        if emotion not in observed_emotions:
            continue
        feature = extract_feature(file, mfcc=True, chroma=True, mel=True)
        x.append(feature)
        y.append(emotion)
    return train_test_split(np.array(x), y, test_size=test_size, random_state=9)

In [ ]:
# Load data and split into training and testing sets
x_train, x_test, y_train, y_test = load_data(test_size=0.2)
print((x_train.shape[0], x_test.shape[0]))  # Print number of samples
print(f'Features extracted: {x_train.shape[1]}')  # Print number of features

(591, 148)
Features extracted: 180


In [ ]:
x_train

array([[-6.27725098e+02,  6.59371643e+01,  9.10654259e+00, ...,
         8.55466260e-06,  9.50507638e-06,  3.72761519e-06],
       [-6.46522766e+02,  5.42023430e+01,  1.08291817e+01, ...,
         8.97055506e-05,  8.28218544e-05,  7.21329634e-05],
       [-3.60190247e+02, -6.44309855e+00, -5.31332588e+01, ...,
         3.65064689e-03,  1.93899486e-03,  1.06055546e-03],
       ...,
       [-5.50096191e+02,  1.70297680e+01, -1.14575634e+01, ...,
         1.51764631e-04,  1.16828531e-04,  8.47479314e-05],
       [-5.62213684e+02,  5.26101494e+01,  1.28255539e+01, ...,
         1.91441679e-04,  1.94725711e-04,  7.13090849e-05],
       [-5.04816345e+02,  3.53618660e+01, -1.43495789e+01, ...,
         6.08151488e-04,  5.55269653e-04,  4.47782222e-04]])

In [ ]:
# Initialize the Multi Layer Perceptron Classifier
model = MLPClassifier(alpha=0.01, batch_size=256, epsilon=1e-08, hidden_layer_sizes=(300,), learning_rate='adaptive', max_iter=500)

In [ ]:
# Train the model
model.fit(x_train, y_train)

MLPClassifier(alpha=0.01, batch_size=256, hidden_layer_sizes=(300,),
              learning_rate='adaptive', max_iter=500)

In [ ]:
#Predict for the test set
y_pred=model.predict(x_test)

In [ ]:
y_pred

array(['fearful', 'disgust', 'calm', 'disgust', 'disgust', 'happy',
       'happy', 'fearful', 'calm', 'calm', 'calm', 'disgust', 'fearful',
       'disgust', 'calm', 'happy', 'happy', 'happy', 'disgust', 'fearful',
       'happy', 'happy', 'calm', 'disgust', 'calm', 'fearful', 'disgust',
       'fearful', 'calm', 'calm', 'fearful', 'calm', 'calm', 'disgust',
       'calm', 'happy', 'happy', 'happy', 'disgust', 'fearful', 'calm',
       'disgust', 'calm', 'fearful', 'happy', 'disgust', 'happy', 'calm',
       'calm', 'happy', 'disgust', 'disgust', 'happy', 'calm', 'calm',
       'calm', 'happy', 'happy', 'happy', 'disgust', 'calm', 'happy',
       'calm', 'calm', 'disgust', 'calm', 'calm', 'fearful', 'fearful',
       'happy', 'calm', 'calm', 'calm', 'disgust', 'calm', 'calm',
       'fearful', 'calm', 'fearful', 'calm', 'happy', 'fearful', 'calm',
       'fearful', 'calm', 'calm', 'calm', 'disgust', 'calm', 'happy',
       'calm', 'calm', 'calm', 'disgust', 'fearful', 'calm', 'calm',


In [ ]:
#Calculate the accuracy of our model
accuracy=accuracy_score(y_true=y_test, y_pred=y_pred)

#Print the accuracy
print("Accuracy: {:.2f}%".format(accuracy*100))

Accuracy: 74.32%


In [ ]:
from sklearn.metrics import accuracy_score, f1_score

In [ ]:
f1_score(y_test, y_pred,average=None)

array([0.81132075, 0.67605634, 0.65217391, 0.76712329])

In [ ]:
import pandas as pd
df=pd.DataFrame({'Actual': y_test, 'Predicted':y_pred})
df.head(10)

,Actual,Predicted
0,fearful,fearful
1,disgust,disgust
2,calm,calm
3,happy,disgust
4,disgust,disgust
5,happy,happy
6,fearful,happy
7,fearful,fearful
8,calm,calm
9,calm,calm


In [ ]:
# Save the model
filename = 'modelForPrediction1.sav'
pickle.dump(model, open(filename, 'wb'))

In [ ]:
# Load the saved model for prediction
loaded_model = pickle.load(open(filename, 'rb'))

In [ ]:
# Audio file for testing
audio_file = "/content/drive/MyDrive/SER21-7/Actor_15/03-01-07-02-02-01-15.wav"

In [ ]:
# Play audio before prediction
print("Playing audio clip for emotion prediction...")
display(Audio(audio_file))

# Extract features for the test audio file
feature = extract_feature(audio_file, mfcc=True, chroma=True, mel=True)
feature = feature.reshape(1, -1)

Playing audio clip for emotion prediction...


In [ ]:
# Predict the emotion
prediction = loaded_model.predict(feature)
print(f"Predicted emotion: {prediction[0]}")

Predicted emotion: disgust
